In [ ]:
import time
import numpy as np
from dotmap import DotMap

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter

import sys
sys.path.append('src')
from read import *
from utils import *
from metrics import *
from model import *
from cpd import *

In [ ]:
name = 'ml'
device = 'cuda:0'
cfg = DotMap()
cfg.dataset = name
cfg.device = device
cfg.rpath = './datasets'
cfg.neg_sampling = 'neg_sample0'

In [ ]:

cfg.rank = 32
cfg.layer_dims = [3, 32, 1]
cfg.depth = len(cfg.layer_dims)
cfg.lr = 1e-3
cfg.wd = 1e-4
cfg.dropout =  0.8
cfg.dropout2 =  0.2
epochs = 1000
cfg.batch_size = 1024


In [ ]:
tensor= read_data(cfg)
cfg.sizes = tensor.sizes
dataset = COODataset(tensor.train_i, tensor.train_v)
dataloader = DataLoader(dataset, batch_size=cfg.batch_size, shuffle=True)

# NeAT

In [ ]:
# create the model
model = NeAT(cfg, cfg.sizes).to(device)
loss_fn = nn.BCELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)
m = nn.Sigmoid()

In [ ]:
flag = 0
err_lst = []
old_valid_acc = 1e+6
# train the model
for epoch in range(epochs):
    start = time.time()
    model.train()
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch[0], batch[1]
        outputs = model(inputs)
        loss = loss_fn(m(outputs), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    model.eval()
    end = time.time()
    if (epoch+1) % 1 == 0:
        with torch.no_grad():
            val_rec = m(model(tensor.valid_i))
            r = eval_(val_rec.data, tensor.valid_v)
            print(f"Epochs {epoch} {end-start:.2}s elapsed || "
                    f"Acc:{r['acc']:.4f} Recall:{r['recall']:.4f}"
                    f"Prec.:{r['prec']:.4f} F1:{r['f1']:.4f} AUC : {r['auc']:.4f}")

            if (old_valid_acc > r['acc']):
                flag +=1
            if flag == 5:
                break
            old_valid_acc = r['acc']

    

In [ ]:
with torch.no_grad():
    val_rec = m(model(tensor.valid_i))
    r = eval_(val_rec.data, tensor.valid_v)
    print(f"Epochs {epoch} {end-start:.2}s elapsed || "
            f"Acc:{r['acc']:.4f} Recall:{r['recall']:.4f}"
            f"Prec.:{r['prec']:.4f} F1:{r['f1']:.4f} AUC : {r['auc']:.4f}")

# CPD casted as classification

In [ ]:
# create the model
model = CPD(cfg).to(device)
loss_fn = nn.BCELoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)
m = nn.Sigmoid()

In [ ]:
flag = 0
err_lst = []
old_valid_acc = 1e+6
# train the model
for epoch in range(epochs):
    start = time.time()
    model.train()
    epoch_loss = 0
    for batch in dataloader:
        optimizer.zero_grad()
        inputs, targets = batch[0], batch[1]
        outputs = model(inputs)
        loss = loss_fn(m(outputs), targets)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    model.eval()
    end = time.time()
    if (epoch+1) % 1 == 0:
        with torch.no_grad():
            val_rec = m(model(tensor.valid_i))
            r = eval_(val_rec.data, tensor.valid_v)
            print(f"Epochs {epoch} {end-start:.2}s elapsed || "
                    f"Acc:{r['acc']:.4f} Recall:{r['recall']:.4f}"
                    f"Prec.:{r['prec']:.4f} F1:{r['f1']:.4f} AUC : {r['auc']:.4f}")

            if (old_valid_acc > r['acc']):
                flag +=1
            if flag == 5:
                break
            old_valid_acc = r['acc']

    

# CPD with recon err.

In [ ]:
cfg.rank = 32
cfg.lr = 1e-2
cfg.wd = 1e-3
epochs = 2000
cfg.batch_size = 1024


In [ ]:
# create the model
model = CPD(cfg).to(device)
optimizer = optim.Adam(model.parameters(), lr=cfg.lr, weight_decay=cfg.wd)
m = nn.Sigmoid()

In [ ]:
flag = 0
err_lst = []
old_valid_rmse = 1e+6
# train the model
for epoch in range(epochs):
    start = time.time()
    optimizer.zero_grad()
    rec = model(tensor.train_i)
    loss = torch.sqrt(((rec - tensor.train_v) ** 2).sum())
    loss.backward()
    optimizer.step()
    end = time.time()
    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            val_rec = model(tensor.valid_i)
            train_rmse = rmse(rec, tensor.train_v)
            valid_rmse = rmse(val_rec, tensor.valid_v)
            print(f"Epochs {epoch} {end-start:.2}s elapsed || "
                    f"TrainRMSE: {train_rmse:.4f}\t"
                    f"ValidRMSE: {valid_rmse:.4f}\t")
                    

            if (old_valid_rmse < valid_rmse):
                flag +=1
            if flag == 10:
                break
            old_valid_rmse = valid_rmse
    

In [ ]:
with torch.no_grad():
    test_rec = model(tensor.test_i)
    r = eval_(test_rec.data, tensor.test_v)
    print(f"Acc:{r['acc']:.4f} Recall:{r['recall']:.4f}"
    f"Prec.:{r['prec']:.4f} F1:{r['f1']:.4f} AUC : {r['auc']:.4f}")